In [1]:
!cd ../code && ./ln_before.sh 9_texture

## 针对 picopng 创建一个 .h

In [2]:
%%file ../code/9_texture/picopng.h
#pragma once
#include <vector>

// args: out RGBA vector; 宽度; 高度; 指向 PNG 文件缓冲区的指针; 大小; 是否要转换为RGBA 32 true 即可
extern int decodePNG(std::vector<unsigned char>& out_image, unsigned long& image_width, unsigned long& image_height, const unsigned char* in_png, size_t in_size, bool convert_to_rgba32 = true);

Overwriting ../code/9_texture/picopng.h


## GLTexture.h

存储加载后 的 PNG

In [3]:
%%file ../code/9_texture/GLTexture.h
#pragma once
#include <GL/glew.h>

struct GLTexture {
    GLuint id;
    int width;
    int height;
};

Overwriting ../code/9_texture/GLTexture.h


## ImageLoader.h ImageLoader.cpp 

加载PNG 到 loadPND 的静态类

In [4]:
%%file ../code/9_texture/ImageLoader.h
#pragma once
#include "GLTexture.h"
#include <string>

class ImageLoader {
public:
    // 从指定目录加载 png 到 GLTexture
    static GLTexture loadPNG(std::string filePath);   
};

Overwriting ../code/9_texture/ImageLoader.h


In [5]:
%%file ../code/9_texture/ImageLoader.cpp
#include "ImageLoader.h"
#include <vector>
#include "picopng.h"
#include "IOManager.h"
#include "Errors.h"

// 从指定目录加载 png 到 GLTexture
GLTexture ImageLoader::loadPNG(std::string filePath) {
    GLTexture texture{};

    std::vector<unsigned char> in;
    std::vector<unsigned char> out;
    unsigned long width, height;

    if (IOManager::readFileToBuffer(filePath, in) == false) {
        fatalError("Failed to load PNG file to buffer!");
        
    }
    
    int errorCode = decodePNG(out, width, height, &in[0], in.size());

    if (errorCode != 0) {
        fatalError("decodePNG failed with error" + std::to_string(errorCode));
    }

    // 分配一个 id
    glGenTextures(1, &(texture.id));

    // 使用这个 texture
    glBindTexture(GL_TEXTURE_2D, texture.id);

    // 实际加载 纹理
    // args: texture 类型; 地图级别; 内部格式; 宽; 高; 是否要边框围绕纹理; 
    //       读取像素数据的格式; 数组类型; 指向数组的指针
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, width, height, 
                 0, GL_RGBA, GL_UNSIGNED_BYTE, &(out[0]));

    // 设置纹理参数 
    // args: texture 类型; 参数名称; 参数值 
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT);
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT);
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR);
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR_MIPMAP_LINEAR);

    glGenerateMipmap(GL_TEXTURE_2D);
    
    glBindTexture(GL_TEXTURE_2D, 0);

    texture.width = width;
    texture.height = height;
    
    return texture;
}

Overwriting ../code/9_texture/ImageLoader.cpp
